# Autor: Jesús Martínez Leal

Fecha de última edición: 24/03/2023

Descripción: Script para visualizar las mallas de los pacientes y las variables seleccionadas por el usuario con la inclusión de widgets.

## Importación de librerías

Las siguientes librerías son necesarias de instalarse para que todo funcione correctamente en el Jupyter Notebook (al menos en Visual Studio Code).

In [1]:
#pip install trame
#pip install vtk
#pip install --upgrade trame-vtk
#pip install --upgrade trame-vuetify

In [2]:
import pyvista as pv
import ipywidgets as widgets

## Código necesario para la visualización 3D

In [3]:
class SetVisibilityCallback:
    
    """Helper callback to keep a reference to the actor being modified."""

    # Constructor de la clase
    
    def __init__(self, actor):
        self.actor = actor

    # Método para cambiar la visibilidad de la malla

    def __call__(self, state):
        self.actor.SetVisibility(state)


# Función de carga de pacientes y visualización de mallas

def cargar_paciente(paciente, idx, var1, var2, plotter):
    
    # Cargar los archivos
    
    path_patient = f'./Pacientes/{paciente}/'
    
    BZS = pv.read(path_patient + 'Border Zone Surface.vtk')
    CS = pv.read(path_patient + 'Core Surface.vtk')
    vT = pv.read(path_patient + 'ventricle_Tagged.vtk')

    # Visualización de partes de ventrículo en cada paciente:
    
    plotter.subplot(0, idx)
    actores = [plotter.add_mesh(mesh, color = color, opacity = 0.5) for mesh, color in zip([BZS, CS, vT], ["blue", "red", "green"])]
    plotter.add_text(f"Paciente {paciente}", position = 'upper_left')

    namesZones = ["BZS", "CS", "vT"]
    text_colors = ["blue", "red", "green"]
    
    # Agregar botones de checkbox para controlar la visibilidad de las mallas
    
    size = 20
    position_y = 12

    for actor, color in zip(actores, ["blue", "red", "green"]):
        callback = SetVisibilityCallback(actor)
        plotter.add_checkbox_button_widget(
            callback,
            value = True,
            position=(5.0, position_y),
            size = size,
            border_size = 1,
            color_on = color,
            color_off = 'grey',
            background_color = 'grey',
        )
        position_y += size + (size // 10)
        
    # Texto para identificar botones
    
    for index, (variable, color) in enumerate(zip(namesZones, text_colors)):

        text = "\n \n" * (index + 1) + variable

        plotter.add_text(text, position='upper_left', font_size = 13, color = color)   
        
        
    # Visualización de la primera variable seleccionada por el usuario   
    
    plotter.subplot(1, idx)
    
    if var1 == "fibers_OR":
        # Campo vectorial (flechas)
        
        centers = vT.points
        directions = vT["fibers_OR"]
        mag = 1
        actor_var1 = plotter.add_arrows(centers, directions, mag = mag, color = "black")
    else:
        # Campo de escalares
        actor_var1 = plotter.add_mesh(vT, scalars = var1, cmap = "viridis")
        
    text_var1 = plotter.add_text(var1, position = 'upper_left', font_size = 13, color = 'blue')
    
    # Visualización de la segunda variable seleccionada por el usuario
    
    plotter.subplot(2, idx)
    
    if var2 == "fibers_OR":
        # Campo vectorial (flechas)
    
        centers = vT.points
        directions = vT["fibers_OR"]
        mag = 1
        actor_var2 = plotter.add_arrows(centers, directions, mag = mag, color = "black")
    else:
        # Campo de escalares
        actor_var2 = plotter.add_mesh(vT, scalars = var2, cmap = "plasma")

    text_var2 = plotter.add_text(var2, position='upper_left', font_size=13, color='red')
    
    return (BZS, CS, vT), (text_var1, text_var2)

# Crear el plotter

def crear_plotter():
    return pv.Plotter(shape=(3, 3))

# Función para actualizar los gráficos cuando se cambian las variables

def actualizar_graficos(var1, var2):
    global added_texts
    plotter = crear_plotter()  # Reconstruir el plotter
    
    added_texts = []  # Restablecer la lista de textos agregados
    
    for idx, paciente in enumerate(pacientes):
        print("Cargando información relacionada con el paciente", paciente)
        meshes, texts = cargar_paciente(paciente, idx, var1, var2, plotter)
        added_texts.extend(texts)  # Agregar los textos a la lista de textos agregados
    
    # Mostrar los nuevos gráficos
    plotter.show()

In [4]:
pacientes = ["p2", "p5", "p36"]

# Variables iniciales

var1 = "scalars"
var2 = "DistEndoToEpi"

In [5]:
# Seguimiento de los textos agregados
added_texts = []

# Widgets para las variables
var1_widget = widgets.Dropdown(
    options=['scalars', 'DistEndoToEpi', 'EndoToEpi', 'Cell_type', 'fibers_OR', 'endo_Norm', 'epi_Norm', 'vector_LAxis', 'tagApexBase', 'DistApexToBase', '17_AHA', '34_pacing'],
    value = var1,
    description='Variable 1:'
)

var2_widget = widgets.Dropdown(
    options=['scalars', 'DistEndoToEpi', 'EndoToEpi', 'Cell_type', 'fibers_OR', 'endo_Norm', 'epi_Norm', 'vector_LAxis', 'tagApexBase', 'DistApexToBase', '17_AHA', '34_pacing'],
    value = var2,
    description='Variable 2:'
)

### Widget para actualizar los gráficos

La primera vez se dispondrán los gráficos en el propio visualizador de Jupyter. A partir de la segunda se abrirá de manera externa en el visualizador usual de Win32 OpenGL (en mi caso).

El hecho de alterar lo de las cajas de selección de variables hace que se vuelva a ejecutar el código de abajo. Puede llevar más o menos tiempo dependiendo de la capacidad de cálculo del dispositivo.

In [6]:
widgets.interactive(
    actualizar_graficos,
    var1 = var1_widget,
    var2 = var2_widget
) 

interactive(children=(Dropdown(description='Variable 1:', options=('scalars', 'DistEndoToEpi', 'EndoToEpi', 'C…